# =============================================================================
# STEP 1: SETUP, IMPORTS, AND REPRODUCIBILITY
# =============================================================================

In [1]:
# Clone the latest repository updates (Augmentation, DataLoaders)
!rm -rf Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication
!git clone https://github.com/trongjhuongwr/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication.git
%cd Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication

Cloning into 'Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication'...
remote: Enumerating objects: 3462, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 3462 (delta 81), reused 104 (delta 45), pack-reused 3297 (from 3)
Receiving objects: 100% (3462/3462), 248.67 MiB | 40.75 MiB/s, done.
Resolving deltas: 100% (386/386), done.
/kaggle/working/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication


In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import json
import shutil
import sys

sys.path.append(os.path.abspath(os.getcwd()))

# Import Custom Modules
from models.feature_extractor import ResNetFeatureExtractor
from models.meta_learner import MetricGenerator
from dataloader.meta_dataloader import SignatureEpisodeDataset
from utils.model_evaluation import evaluate_and_plot, visualize_hard_examples, compute_metrics, compute_metrics, _plot_det_curve, _plot_far_frr, _plot_confusion_matrix, _plot_score_distribution, _plot_roc_curve

# Deterministic Seeding for Reproducible Research
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f" > [System] Seed set to: {seed}")

seed_everything(42)

# Device Configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f" > [System] Computation Device: {DEVICE}")

 > [System] Seed set to: 42
 > [System] Computation Device: cuda


# =============================================================================
# STEP 2: ROBUST DATA ACQUISITION & PREPROCESSING
# =============================================================================

In [3]:
# 1. Setup Directories
WORKING_DIR = '/kaggle/working'
SPLIT_DIR = os.path.join(WORKING_DIR, 'splits')
GENUINE_DIR = os.path.join(WORKING_DIR, 'all_genuine')
FORGED_DIR = os.path.join(WORKING_DIR, 'all_forged')

for d in [GENUINE_DIR, FORGED_DIR, SPLIT_DIR]:
    if os.path.exists(d): shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)

# 2. Link Data Again (Because /kaggle/working is wiped in new session)
print(" > Linking dataset...")
!ln -sf /kaggle/input/bhsig260-hindi-bengali/BHSig160_Hindi/Genuine/* {GENUINE_DIR}/
!ln -sf /kaggle/input/bhsig260-hindi-bengali/BHSig160_Hindi/Genuine/* {FORGED_DIR}/
!ln -sf /kaggle/input/bhsig260-hindi-bengali/BHSig100_Bengali/Genuine/* {GENUINE_DIR}/
!ln -sf /kaggle/input/bhsig260-hindi-bengali/BHSig100_Bengali/Forged/* {FORGED_DIR}/

# 3. Run Restructure Script
print(" > Generating Folds...")
!python scripts/restructure_bhsig.py \
    --base_dir /kaggle/input/bhsig260-hindi-bengali \
    --output_dir /kaggle/working/splits \
    --pretrain_users 150
    
print("Data Setup Complete for Meta-Training.")

 > Linking dataset...
 > Generating Folds...
[Info] Scanning data structure at: /kaggle/input/bhsig260-hindi-bengali
[Info] Found 14040 image files. Parsing metadata...
[Info] Successfully validated 260 users with complete data.
[Info] Split Statistics:
   - Background Users (Pre-training): 150
   - Evaluation Users (Meta-learning): 110
[Success] Saved pre-training split to: /kaggle/working/splits/bhsig_background_users.json
   > Generated Fold 0: 88 Train / 22 Val users.
   > Generated Fold 1: 88 Train / 22 Val users.
   > Generated Fold 2: 88 Train / 22 Val users.
   > Generated Fold 3: 88 Train / 22 Val users.
   > Generated Fold 4: 88 Train / 22 Val users.
Data Setup Complete for Meta-Training.


# =============================================================================
# STEP 3: HYPERPARAMETER CONFIGURATION
# =============================================================================

In [4]:
# --- Input Configuration ---
IMG_SIZE = 224 
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE)

# --- Meta-Learning Protocol ---
# N-way K-shot configuration (1-way 1-shot for Verification)
N_WAY = 1               # Binary Classification (Same/Diff)
K_SHOT = 1              # Number of reference signatures (Support Set)
N_QUERY_GENUINE = 1     # Positive samples in Query Set
N_QUERY_FORGERY = 1     # Negative samples in Query Set

# --- Training Hyperparameters ---
BATCH_SIZE = 16         # Smaller batch size for Meta-learning stability
MAX_EPOCHS = 60         # Sufficient for convergence with fine-tuning
LR_STAGE_2 = 1e-5       # Very low learning rate to preserve pre-trained knowledge

# --- Paths ---
PRETRAINED_WEIGHTS_PATH = '/kaggle/input/my-pretrained-weights/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/background_pretrain.pth' 

if not os.path.exists(PRETRAINED_WEIGHTS_PATH):
    print(f"WARNING: Pretrained weights not found at {PRETRAINED_WEIGHTS_PATH}")
    print("Please upload the 'background_pretrain.pth' file and update the path.")
else:
    print(f"FOUND PRETRAINED WEIGHTS: {PRETRAINED_WEIGHTS_PATH}")

FOUND PRETRAINED WEIGHTS: /kaggle/input/my-pretrained-weights/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/background_pretrain.pth


# =============================================================================
# STEP 4: TRAINING UTILITIES
# =============================================================================

In [5]:
def initialize_models(device, pretrained_path=None):
    """
    Initializes the Siamese Feature Extractor and the Metric Generator.
    """
    feature_extractor = ResNetFeatureExtractor(backbone_name='resnet34').to(device)
    
    if pretrained_path and os.path.exists(pretrained_path):
        print(f"   ... Loading pre-trained weights from {os.path.basename(pretrained_path)}")
        state_dict = torch.load(pretrained_path, map_location=device)
        feature_extractor.load_state_dict(state_dict)
    else:
        print("   ... Warning: Initializing with ImageNet weights (Suboptimal).")

    metric_generator = MetricGenerator(embedding_dim=1024).to(device)
    
    return feature_extractor, metric_generator

def train_epoch(feature_extractor, metric_generator, dataloader, optimizer, criterion, device):
    feature_extractor.train()
    metric_generator.train()
    
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc="Training", leave=False)
    
    for batch in pbar:
        # Unpack Data
        support_imgs = batch['support_images'].squeeze(1).to(device) 
        query_imgs = batch['query_images'].to(device)
        labels = batch['query_labels'].to(device)
        
        # Flatten setup
        B, N_Q, C, H, W = query_imgs.shape
        query_imgs_flat = query_imgs.view(B * N_Q, C, H, W)
        labels_flat = labels.view(B * N_Q).unsqueeze(1) # [B, 1]
        
        support_imgs_expanded = support_imgs.unsqueeze(1).expand(-1, N_Q, -1, -1, -1)
        support_imgs_flat = support_imgs_expanded.reshape(B * N_Q, C, H, W)
        
        optimizer.zero_grad()
        
        # Forward Pass
        support_feats = feature_extractor(support_imgs_flat)
        query_feats = feature_extractor(query_imgs_flat)
        combined_feats = torch.cat((support_feats, query_feats), dim=1)
        
        # Tính Logits
        scores = metric_generator(combined_feats)
        loss = criterion(scores, labels_flat)
        
        # Backward Pass
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Calculate Accuracy (Convert Logits -> Probability -> Prediction (Threshold 0.5))
        preds = (torch.sigmoid(scores) > 0.5).float()
        correct += (preds == labels_flat).sum().item()
        total += labels_flat.size(0)
        
        pbar.set_postfix({'loss': loss.item()})
        
    avg_loss = running_loss / len(dataloader)
    avg_acc = correct / total if total > 0 else 0.0
    
    return avg_loss, avg_acc

def validate_epoch(feature_extractor, metric_generator, dataloader, device):
    """
    Validation Loop: Calculates EER and Accuracy using compute_metrics.
    """
    feature_extractor.eval()
    metric_generator.eval()
    
    all_labels = []
    all_scores = [] 
    
    with torch.no_grad():
        for batch in dataloader:
            support_imgs = batch['support_images'].squeeze(1).to(device)
            query_imgs = batch['query_images'].to(device)
            labels = batch['query_labels'].to(device)
            
            B, N_Q, C, H, W = query_imgs.shape
            query_imgs_flat = query_imgs.view(B * N_Q, C, H, W)
            labels_flat = labels.view(B * N_Q).unsqueeze(1)
            
            support_imgs_flat = support_imgs.unsqueeze(1).expand(-1, N_Q, -1, -1, -1).reshape(B * N_Q, C, H, W)
            
            s_feats = feature_extractor(support_imgs_flat)
            q_feats = feature_extractor(query_imgs_flat)
            combined = torch.cat((s_feats, q_feats), dim=1)
            
            scores = metric_generator(combined)
            probs = torch.sigmoid(scores)

            all_scores.extend(probs.cpu().numpy().flatten())
            all_labels.extend(labels_flat.cpu().numpy().flatten())
            
    metrics = compute_metrics(all_labels, all_scores)
    return metrics

# =============================================================================
# STEP 5: 5-FOLD CROSS-VALIDATION EXECUTION
# =============================================================================

In [6]:
N_FOLDS = 5
results = {}
CHECKPOINT_DIR = './checkpoints_meta'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print(f"Starting {N_FOLDS}-Fold Cross-Validation...")
print(f"Using Pre-trained Weights: {PRETRAINED_WEIGHTS_PATH}")

fold_results = []
for fold in range(N_FOLDS):
    print(f"\n{'-'*40}")
    print(f"FOLD {fold + 1}/{N_FOLDS}")
    print(f"{'-'*40}")
    
    # 1. DATALOADER SETUP
    # Define split files for this fold
    train_split = os.path.join(SPLIT_DIR, f'bhsig_meta_split_fold_{fold}.json')
    
    # Load Train Set (Augmentation Enabled for robustness)
    train_set = SignatureEpisodeDataset(
        train_split, root_dir=WORKING_DIR, mode='train', 
        k_shot=K_SHOT, n_query_genuine=N_QUERY_GENUINE, n_query_forgery=N_QUERY_FORGERY,
        augment=True, use_full_path=True # ENABLE AUGMENTATION
    )
    
    # Load Val Set (Augmentation Disabled for deterministic eval)
    val_set = SignatureEpisodeDataset(
        train_split, root_dir=WORKING_DIR, mode='val',
        k_shot=K_SHOT, n_query_genuine=N_QUERY_GENUINE, n_query_forgery=N_QUERY_FORGERY,
        augment=False, use_full_path=True
    )
    
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    # 2. MODEL INITIALIZATION
    feature_extractor, metric_generator = initialize_models(DEVICE, PRETRAINED_WEIGHTS_PATH)
    
    # Optimizer
    optimizer = optim.AdamW([
        {'params': feature_extractor.parameters(), 'lr': LR_STAGE_2},
        {'params': metric_generator.parameters(), 'lr': LR_STAGE_2}
    ], weight_decay=1e-4)
    
    criterion = nn.BCEWithLogitsLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    # 3. TRAINING LOOP
    best_eer = 1.0
    best_acc = 0.0
    best_metrics = {}
    
    for epoch in range(MAX_EPOCHS):
        # Train
        train_loss, train_acc = train_epoch(feature_extractor, metric_generator, train_loader, optimizer, criterion, DEVICE)
        
        # Validation (Calculate EER and Accuracy)
        val_metrics = validate_epoch(feature_extractor, metric_generator, val_loader, DEVICE)
        val_eer = val_metrics['eer']
        val_acc = val_metrics['accuracy']
        
        # Logging
        print(f"Epoch {epoch+1:02d} | Loss: {train_loss:.4f} | Train Acc: {train_acc:.2%} | Val EER: {val_eer:.2%} | Val Acc: {val_acc:.2%}")
        
        scheduler.step(val_eer)
        
        # Save Best Model Logic (Minimize EER)
        if val_eer < best_eer:
            best_eer = val_eer
            best_acc = val_acc
            best_metrics = val_metrics

            # Save Checkpoint
            ckpt_path = os.path.join(CHECKPOINT_DIR, f"best_model_fold_{fold}.pth")
            torch.save({
                'fold': fold,
                'epoch': epoch,
                'feature_extractor': feature_extractor.state_dict(),
                'metric_generator': metric_generator.state_dict(),
                'val_eer': val_eer,
                'val_acc': val_acc
            }, ckpt_path)
            print(f"   >>> New Best Model Saved! (EER: {val_eer:.2%} | ACC: {val_acc:.2%})")
        
        elif val_eer == best_eer and val_acc > best_acc:
            best_acc = val_acc
            best_metrics = val_metrics
            
            # Save Checkpoint
            ckpt_path = os.path.join(CHECKPOINT_DIR, f"best_model_fold_{fold}.pth")
            torch.save({
                'fold': fold,
                'epoch': epoch,
                'feature_extractor': feature_extractor.state_dict(),
                'metric_generator': metric_generator.state_dict(),
                'val_eer': val_eer,
                'val_acc': val_acc
            }, ckpt_path)
            print(f"   >>> New Best Model Saved! (EER: {val_eer:.2%} | ACC: {val_acc:.2%})")

    # Record Fold Results
    results[fold] = {'best_eer': best_eer, 'best_acc': best_acc}
    print(f"Fold {fold+1} Completed. Best EER: {best_eer:.2%}")
    fold_results.append(best_metrics)

Starting 5-Fold Cross-Validation...
Using Pre-trained Weights: /kaggle/input/my-pretrained-weights/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication/background_pretrain.pth

----------------------------------------
FOLD 1/5
----------------------------------------
 > [Dataset] Loaded subset 'meta-train' with 88 users.
 > [Dataset] Loaded subset 'meta-test' with 22 users.
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


100%|██████████| 83.3M/83.3M [00:00<00:00, 241MB/s]


   ... Loading pre-trained weights from background_pretrain.pth


Training:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 01 | Loss: 0.6641 | Train Acc: 63.07% | Val EER: 27.27% | Val Acc: 75.00%
   >>> New Best Model Saved! (EER: 27.27% | ACC: 75.00%)


...

Epoch 58 | Loss: 0.3279 | Train Acc: 86.93% | Val EER: 13.64% | Val Acc: 86.36%


Training:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 59 | Loss: 0.3171 | Train Acc: 87.50% | Val EER: 9.09% | Val Acc: 88.64%


Training:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 60 | Loss: 0.3462 | Train Acc: 82.95% | Val EER: 13.64% | Val Acc: 88.64%
Fold 5 Completed. Best EER: 9.09%


# =============================================================================
# STEP 6: FINAL REPORT
# =============================================================================

In [7]:
print(f"\n{'='*40}")
print(f"{'FINAL CROSS-VALIDATION RESULTS':^40}")
print(f"{'='*40}")

# Khởi tạo các list để tính trung bình
avg_metrics = {
    'eer': [], 'accuracy': [], 'auc': [], 
    'f1': [], 'precision': [], 'recall': []
}

# Duyệt qua kết quả từng fold
# LƯU Ý: Đảm bảo tên biến 'fold_results' trùng với tên biến list bạn dùng trong vòng lặp
for i, result in enumerate(fold_results):
    print(f"Fold {i+1}: "
          f"EER = {result['eer']:.2%} | "
          f"Acc = {result['accuracy']:.2%} | "
          f"AUC = {result['auc']:.4f} | "
          f"F1 = {result['f1']:.4f} | "
          f"Prec = {result['precision']:.4f} | "
          f"Rec = {result['recall']:.4f}")
    
    # Lưu giá trị để tính mean
    for key in avg_metrics.keys():
        avg_metrics[key].append(result[key])

print("-" * 40)
print(f"AVERAGE EER       : {np.mean(avg_metrics['eer']):.2%}")
print(f"AVERAGE ACCURACY  : {np.mean(avg_metrics['accuracy']):.2%}")
print(f"AVERAGE AUC       : {np.mean(avg_metrics['auc']):.4f}")
print(f"AVERAGE F1-SCORE  : {np.mean(avg_metrics['f1']):.4f}")
print(f"AVERAGE PRECISION : {np.mean(avg_metrics['precision']):.4f}")
print(f"AVERAGE RECALL    : {np.mean(avg_metrics['recall']):.4f}")
print("=" * 40)


     FINAL CROSS-VALIDATION RESULTS     
Fold 1: EER = 4.55% | Acc = 95.45% | AUC = 0.9855 | F1 = 0.9545 | Prec = 0.9545 | Rec = 0.9545
Fold 2: EER = 9.09% | Acc = 90.91% | AUC = 0.9545 | F1 = 0.9091 | Prec = 0.9091 | Rec = 0.9091
Fold 3: EER = 0.00% | Acc = 100.00% | AUC = 1.0000 | F1 = 1.0000 | Prec = 1.0000 | Rec = 1.0000
Fold 4: EER = 18.18% | Acc = 84.09% | AUC = 0.8678 | F1 = 0.8444 | Prec = 0.8261 | Rec = 0.8636
Fold 5: EER = 9.09% | Acc = 90.91% | AUC = 0.9256 | F1 = 0.9091 | Prec = 0.9091 | Rec = 0.9091
----------------------------------------
AVERAGE EER       : 8.18%
AVERAGE ACCURACY  : 92.27%
AVERAGE AUC       : 0.9467
AVERAGE F1-SCORE  : 0.9234
AVERAGE PRECISION : 0.9198
AVERAGE RECALL    : 0.9273


In [8]:
# print("\nGenerating Detailed Report for Fold 1...")
# feature_extractor, metric_generator = initialize_models(DEVICE)

# # Load best model
# ckpt = torch.load(f"{CHECKPOINT_DIR}/best_model_fold_0.pth", weights_only=False)
# feature_extractor.load_state_dict(ckpt['feature_extractor'])
# metric_generator.load_state_dict(ckpt['metric_generator'])

# # Generate DataLoader
# val_split_file = os.path.join(SPLIT_DIR, 'bhsig_meta_split_fold_0.json')
# val_set = SignatureEpisodeDataset(val_split_file, root_dir=WORKING_DIR, mode='val', k_shot=1, augment=False, use_full_path=True)
# val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

# # Visualization
# evaluate_and_plot(feature_extractor, metric_generator, val_loader, DEVICE, save_dir='./final_report_fold_0')

In [9]:
# print("\nGenerating Detailed Report for Fold 2...")
# feature_extractor, metric_generator = initialize_models(DEVICE)

# # Load best model
# ckpt = torch.load(f"{CHECKPOINT_DIR}/best_model_fold_1.pth", weights_only=False)
# feature_extractor.load_state_dict(ckpt['feature_extractor'])
# metric_generator.load_state_dict(ckpt['metric_generator'])

# # Generate DataLoader
# val_split_file = os.path.join(SPLIT_DIR, 'bhsig_meta_split_fold_1.json')
# val_set = SignatureEpisodeDataset(val_split_file, root_dir=WORKING_DIR, mode='val', k_shot=1, augment=False, use_full_path=True)
# val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

# # Visualization
# evaluate_and_plot(feature_extractor, metric_generator, val_loader, DEVICE, save_dir='./final_report_fold_1')

In [10]:
# print("\nGenerating Detailed Report for Fold 3...")
# feature_extractor, metric_generator = initialize_models(DEVICE)

# # Load best model
# ckpt = torch.load(f"{CHECKPOINT_DIR}/best_model_fold_2.pth", weights_only=False)
# feature_extractor.load_state_dict(ckpt['feature_extractor'])
# metric_generator.load_state_dict(ckpt['metric_generator'])

# # Generate DataLoader
# val_split_file = os.path.join(SPLIT_DIR, 'bhsig_meta_split_fold_2.json')
# val_set = SignatureEpisodeDataset(val_split_file, root_dir=WORKING_DIR, mode='val', k_shot=1, augment=False, use_full_path=True)
# val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

# # Visualization
# evaluate_and_plot(feature_extractor, metric_generator, val_loader, DEVICE, save_dir='./final_report_fold_2')

In [11]:
# print("\nGenerating Detailed Report for Fold 4...")
# feature_extractor, metric_generator = initialize_models(DEVICE)

# # Load best model
# ckpt = torch.load(f"{CHECKPOINT_DIR}/best_model_fold_3.pth", weights_only=False)
# feature_extractor.load_state_dict(ckpt['feature_extractor'])
# metric_generator.load_state_dict(ckpt['metric_generator'])

# # Generate DataLoader
# val_split_file = os.path.join(SPLIT_DIR, 'bhsig_meta_split_fold_3.json')
# val_set = SignatureEpisodeDataset(val_split_file, root_dir=WORKING_DIR, mode='val', k_shot=1, augment=False, use_full_path=True)
# val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

# # Visualization
# evaluate_and_plot(feature_extractor, metric_generator, val_loader, DEVICE, save_dir='./final_report_fold_3')

In [12]:
# print("\nGenerating Detailed Report for Fold 5...")
# feature_extractor, metric_generator = initialize_models(DEVICE)

# # Load best model
# ckpt = torch.load(f"{CHECKPOINT_DIR}/best_model_fold_4.pth", weights_only=False))
# feature_extractor.load_state_dict(ckpt['feature_extractor'])
# metric_generator.load_state_dict(ckpt['metric_generator'])

# # Generate DataLoader
# val_split_file = os.path.join(SPLIT_DIR, 'bhsig_meta_split_fold_4.json')
# val_set = SignatureEpisodeDataset(val_split_file, root_dir=WORKING_DIR, mode='val', k_shot=1, augment=False, use_full_path=True)
# val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

# # Visualization
# evaluate_and_plot(feature_extractor, metric_generator, val_loader, DEVICE, save_dir='./final_report_fold_4')